In [8]:
# Load libraries
import requests
import re
from pandas import read_csv
from pandas import read_html
from pandas import read_json
from pandas import read_excel
from pandas import read_stata
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [26]:
read_csv("/Volumes/GoogleDrive/My Drive/Market Insights Library/04_Thought Leadership Ideas/Cross-Client Analytics/CWS Benchmarks/CWS Indicators_Aggr.csv")

,Client Name,ga:date,ga:region,ga:userType,ga:channelGrouping,ga:deviceCategory,ga:pagePathLevel1,ga:userAgeBracket,ga:userGender,ga:users,ga:sessions,view_id,ga:pageviews,ga:bounces,ga:sessionDuration (Sec),ga:ApplAtt,ga:entrances,ga:exits,ga:timeOnPage,Table Source
0,ALLSTATE,4/6/2019,Oregon,New Visitor,Direct,Desktop,NaN,NaN,NaN,1.0,1.0,178916158,2,0,1.0,0.0,NaN,NaN,NaN,2019_conversion_apr.csv
1,BON SECOURS,9/27/2019,North Carolina,Returning Visitor,Direct,Tablet,Job Description,NaN,NaN,NaN,NaN,179314322,1,1,NaN,NaN,1.0,1.0,0.0,2019_content_sep.csv
2,HILTON WORLDWIDE,7/18/2019,Other,New Visitor,Direct,Mobile,All Other Pages,NaN,NaN,NaN,NaN,73615399,1,1,NaN,NaN,1.0,1.0,0.0,2019_content_jul.csv
3,THE TRAVELERS COMPANIES INC,8/21/2019,England,New Visitor,Social,Desktop,Job Description,NaN,NaN,NaN,NaN,179682380,2,0,NaN,NaN,2.0,0.0,13.0,2019_content_aug.csv
4,MEMORIAL HEALTH SYSTEM,11/7/2019,Florida,Returning Visitor,Social,Mobile,NaN,NaN,NaN,12.0,14.0,179628091,57,7,2040.0,3.0,NaN,NaN,NaN,2019_conversion_nov.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35550063,CARMAX,5/9/2019,Colorado,Returning Visitor,Paid Search,Mobile,All Other Pages,NaN,NaN,NaN,NaN,179447555,1,1,NaN,NaN,1.0,1.0,0.0,2019_content_may.csv
35550064,THE BEAUTIFUL GROUP,7/19/2019,Ohio,New Visitor,Referral,Mobile,All Other Pages,NaN,NaN,NaN,NaN,182277462,1,0,NaN,NaN,0.0,0.0,14.0,2019_content_jul.csv
35550065,HILTON WORLDWIDE,9/22/2019,Other,Returning Visitor,Organic Search,Desktop,All Other Pages,NaN,NaN,NaN,NaN,73615399,3,0,NaN,NaN,0.0,0.0,29.0,2019_content_sep.csv
35550066,DICKS SPORTING GOODS,6/18/2020,Georgia,New Visitor,Referral,Tablet,NaN,NaN,NaN,4.0,4.0,180762565,5,2,346.0,1.0,NaN,NaN,NaN,2020_conversion_jun.csv


In [25]:
# Read online dataset directly into DataFrame
url = "https://www.fdic.gov/bank/individual/failed/banklist.html"
dataset = read_html(url)
print(dataset)

[                             Bank Name           City  ST   CERT  \
0                 The First State Bank  Barboursville  WV  14361   
1                   Ericson State Bank        Ericson  NE  18265   
2     City National Bank of New Jersey         Newark  NJ  21111   
3                        Resolute Bank         Maumee  OH  58317   
4                Louisa Community Bank         Louisa  KY  58112   
..                                 ...            ...  ..    ...   
556                 Superior Bank, FSB       Hinsdale  IL  32646   
557                Malta National Bank          Malta  OH   6629   
558    First Alliance Bank & Trust Co.     Manchester  NH  34264   
559  National State Bank of Metropolis     Metropolis  IL   3815   
560                   Bank of Honolulu       Honolulu  HI  21029   

                   Acquiring Institution       Closing Date  
0                         MVB Bank, Inc.      April 3, 2020  
1             Farmers and Merchants Bank  February 14, 202

In [ ]:
# Describe & visualize data
# box and whisker plots
dataset.plot(kind='box', subplots=True, layout=(2,2), sharex=False, sharey=False)
pyplot.show()
# histograms
dataset.hist()
pyplot.show()
# scatter plot matrix
scatter_matrix(dataset)
pyplot.show()

In [ ]:
# Split into training & validation
array = dataset.values
X = array[:,0:4]
Y = array[:,4]
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.20, random_state=1)


In [ ]:
# Make predictions on validation dataset - SVC
modelSVC = SVC(gamma='auto')
modelSVC.fit(X_train, Y_train)
predictions = modelSVC.predict(X_validation)

# Evaluate predictions
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

In [ ]:
# Make predictions on validation dataset - Logistic
modelCART = DecisionTreeClassifier()
modelCART.fit(X_train, Y_train)
predictions = modelCART.predict(X_validation)

# Evaluate predictions
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

In [ ]:
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))


In [ ]:
# Evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))